<a href="https://colab.research.google.com/github/ProfEddie/HypCLIP/blob/main/HypCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/alex-tifrea/poincare_glove.git
!git clone https://github.com/facebookresearch/poincare-embeddings.git

ERROR: unknown command "isntall" - maybe you meant "install"
Cloning into 'poincare_glove'...
remote: Enumerating objects: 703, done.
remote: Total 703 (delta 0), reused 0 (delta 0), pack-reused 703
Receiving objects: 100% (703/703), 63.85 MiB | 17.67 MiB/s, done.
Resolving deltas: 100% (98/98), done.
Cloning into 'poincare-embeddings'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 157 (delta 9), reused 19 (delta 9), pack-reused 137
Receiving objects: 100% (157/157), 399.47 KiB | 11.75 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [ ]:
!python /content/poincare-embeddings/setup.py build_ext --inplace

In [ ]:
%cd /content/poincare-embeddings/wordnet
!python transitive_closure.py

In [ ]:
!pip install iopath
!pip install git+https://github.com/facebookresearch/hypernymysuite.git

In [ ]:
%cd /content/poincare-embeddings/
!./train-nouns.sh -d 100 -m poincare

In [5]:
!pip install transformers
!pip install datasets

In [6]:
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor
import itertools
from datasets import dataset_dict

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
model_ckt = 'openai/clip-vit-base-patch32'
processor = CLIPProcessor.from_pretrained(model_ckt)

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=333x500 at 0x795A9F133850>,
 'caption': ['Two young guys with shaggy hair look at their hands while hanging out in the yard.',
  'Two young, White males are outside near many bushes.',
  'Two men in green shirts are standing in a yard.',
  'A man in a blue shirt standing in a garden.',
  'Two friends enjoy time spent together.'],
 'sentids': ['0', '1', '2', '3', '4'],
 'split': 'train',
 'img_id': '0',
 'filename': '1000092795.jpg'}

In [35]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:

flickr30k = load_dataset('EddieChen372/flickr30k')

In [15]:
flickr30k

DatasetDict({
    train: Dataset({
        features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename', 'input_ids', 'attention_mask', 'pixel_values'],
        num_rows: 29000
    })
    test: Dataset({
        features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename', 'input_ids', 'attention_mask', 'pixel_values'],
        num_rows: 1014
    })
    val: Dataset({
        features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename', 'input_ids', 'attention_mask', 'pixel_values'],
        num_rows: 1000
    })
})

In [16]:

class Flickr_dataset(Dataset):
    def __init__(self, config, dataset, processor  ):
        self.device = torch.device(config['device'])
        self.dataset = dataset
        self.processor = processor
        self.dataset_len = len(dataset)
        self.cap_per_img = 5


    def __len__(self):
        return len(self.dataset)  * self.cap_per_img

    def __getitem__(self, index):
        data = self.dataset[int(index / self.cap_per_img)]
        out = {
            'img_id': data['img_id'],
            'input_ids': data['input_ids'][int(index % self.cap_per_img)],
            'attention_mask': data['attention_mask'][(index % self.cap_per_img)],
            'pixel_values': data['pixel_values'],
            'image': data['image'],
            'caption': data['caption'][int(index % self.cap_per_img)],
            'sent_id': data['sentids'][int(index % self.cap_per_img)]
        }
        return out

config = {
    'device': 'cuda'
}
train =  Flickr_dataset(config, flickr30k['train'], processor)
test =  Flickr_dataset(config, flickr30k['test'], processor)
val =  Flickr_dataset(config, flickr30k['val'], processor)



In [20]:
print(torch.tensor(train[23]['pixel_values']).shape)
print(torch.tensor(train[len(train)-1]['input_ids']).shape)
print(torch.tensor(train[len(train)-1]['attention_mask']).shape)

torch.Size([1, 3, 224, 224])
torch.Size([20])
torch.Size([20])


In [21]:
import pandas as pd
import numpy as np
from transformers import CLIPTokenizerFast

tokenizer = CLIPTokenizerFast.from_pretrained('openai/clip-vit-base-patch32')

In [22]:
tokenizer.pad_token_id

49407

In [29]:
from itertools import zip_longest

def collate_func(batch):
  df = pd.DataFrame(batch)
  return processor(text=list(df['caption']), images=list(df['image']),padding=True, return_tensors='pt')



In [30]:
train_loader = DataLoader(train, batch_size=40, collate_fn=collate_func ,shuffle=True  )
test_loader = DataLoader(test, batch_size=40, collate_fn=collate_func ,shuffle=True  )
val_loader = DataLoader(val, batch_size=40, collate_fn=collate_func ,shuffle=True  )

In [33]:
from tqdm.auto import tqdm
progress = tqdm(range(len(train_loader)))
for data in train_loader:
  print(data['input_ids'].shape)
  print(data['attention_mask'].shape)
  print(data['pixel_values'].shape)
  progress.update(1)
  break


  0%|          | 0/3625 [00:00<?, ?it/s]

torch.Size([40, 25])
torch.Size([40, 25])
torch.Size([40, 3, 224, 224])
